# Model building and training

In [2]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [3]:
# Initializing the CNN model
classifier = Sequential()

In [4]:
# Adding CNN layers
# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the layers
classifier.add(Flatten())

In [5]:
# Adding Dense layers
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax')) # softmax for more than 2
#summary of our model
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [6]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2

In [7]:
#importing imageDataGenerator library
from keras.preprocessing.image import ImageDataGenerator

In [8]:
#configuring ImageDataGenerator class
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='qXS1mQhX1ApIWKXsA-TLQoZcNkEsvriGdjRfyWxyMap8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasedtoolforsterilebrowsi-donotdelete-pr-jm0lbmb8virp7c'
object_key = 'dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [10]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [11]:
import os
os.listdir('./dataset')

['test', 'train']

In [12]:
#Applying ImageDataGenerator Functionality To Trainset
training_set = train_datagen.flow_from_directory('./dataset/train',
                                                 target_size=(64, 64),
                                                 batch_size=5,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

#Applying ImageDataGenerator Functionality To Testset
test_set = test_datagen.flow_from_directory('./dataset/test',
                                            target_size=(64, 64),
                                            batch_size=5,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

Found 600 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [16]:
# Training the model
model = classifier.fit(
        training_set,
        epochs=10,
        validation_data=test_set)

Epoch 1/10
120/120 [==============================] - 2s 18ms/step - loss: 0.1004 - accuracy: 0.9617 - val_loss: 0.2489 - val_accuracy: 0.9333
Epoch 2/10
120/120 [==============================] - 2s 19ms/step - loss: 0.0824 - accuracy: 0.9717 - val_loss: 0.3255 - val_accuracy: 0.8667
Epoch 3/10
120/120 [==============================] - 2s 18ms/step - loss: 0.0496 - accuracy: 0.9817 - val_loss: 0.4066 - val_accuracy: 0.9333
Epoch 4/10
120/120 [==============================] - 2s 19ms/step - loss: 0.0450 - accuracy: 0.9833 - val_loss: 0.2316 - val_accuracy: 0.9667
Epoch 5/10
120/120 [==============================] - 2s 19ms/step - loss: 0.0466 - accuracy: 0.9833 - val_loss: 0.4857 - val_accuracy: 0.9333
Epoch 6/10
120/120 [==============================] - 2s 19ms/step - loss: 0.0451 - accuracy: 0.9883 - val_loss: 0.3834 - val_accuracy: 0.9333
Epoch 7/10
120/120 [==============================] - 2s 19ms/step - loss: 0.0336 - accuracy: 0.9883 - val_loss: 0.3412 - val_accuracy: 0.9667

In [19]:
# Saving the model
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
classifier.save_weights('model.h5')

# Model deployment

In [20]:
!tar -zcvf model.tgz model.h5

model.h5


In [21]:
ls -1

dataset/
model.h5
model.json
model.tgz


In [22]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 22.7 MB/s eta 0:00:01


In [23]:
!pip install ibm_watson_machine_learning

In [24]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"YAMSXTX00wm2b0fwGobW29vtjdBjHw7vNZ5W1kHCQBYU"
}
client = APIClient(wml_credentials)
client

In [25]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/49cea5f263ee472d96fb28ba348fe70a:2322d2b8-d298-4775-af1f-31a041ed933e::',
      'guid': '2322d2b8-d298-4775-af1f-31a041ed933e',
      'name': 'Watson Machine Learning-wh',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'A Gesture-based Tool for Sterile  Browsing of Radiology Images',
    'scope': {'bss_account_id': '49cea5f263ee472d96fb28ba348fe70a'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '5293e8ff-372b-491a-85d0-056d479c1585',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '432c9e33699541fabba2002b1571be18',
        'api_key': 'KExtkgrBRFrvcD8EI54VM11PbOVHrb2bCcBhjVJOHOVI',
        'secret_access_key': 'f87b22ddc1973f0365e16d0040350f73c5cd2c7d094ae8af',
        'service_id': 'ServiceId-049208ee-e26e-4de6-9350-e3d0c5d0537d'},
       'editor': {'access_key_id': 'f

In [26]:
space_id = '9f30bada-ef46-4279-931f-c71e12bca948'

In [27]:
client.set.default_space(space_id)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [30]:
model_details = client.repository.store_model(model='model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME : 'Hand gesture prediction model',
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-18T13:38:57.283Z',
  'id': '40874cfe-7098-4313-9b25-8b7430dbdaf4',
  'modified_at': '2022-11-18T13:39:00.136Z',
  'name': 'Hand gesture prediction model',
  'owner': 'IBMid-6630042TEN',
  'resource_key': '8616f538-0f97-4c05-8a7b-e37a86edea9b',
  'space_id': '9f30bada-ef46-4279-931f-c71e12bca948'},
 'system': {'warnings': []}}

In [31]:
model_id = client.repository.get_model_id(model_details)
model_id

'40874cfe-7098-4313-9b25-8b7430dbdaf4'

# Downloading the deployed model

In [32]:
client.repository.download(model_id,'model.tar.gb')

Successfully saved model content to file: 'model.tar.gb'


'/home/wsuser/work/model.tar.gb'

In [33]:
ls -1

dataset/
model.h5
model.json
model.tar.gb
model.tgz
